<a href="https://colab.research.google.com/github/AnggelDwiP/Data-Mining_SIB-2A/blob/main/modul12_latihan3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score

# Load dataset Titanic
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
data = pd.read_csv(url)

# Data Preprocessing
# Fill missing values
data['Age'].fillna(data['Age'].mean(), inplace=True)
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)
data['Fare'].fillna(data['Fare'].mean(), inplace=True)

# Drop columns that won't be used
data.drop(['Name', 'Ticket', 'Cabin', 'PassengerId'], axis=1, inplace=True)

# Convert categorical columns to numerical values
data = pd.get_dummies(data, columns=['Sex', 'Embarked'], drop_first=True)

# Separate features and labels
X = data.drop('Survived', axis=1)
y = data['Survived']

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# One-hot encode labels
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y.values.reshape(-1, 1))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define activation functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def relu(x):
    return np.maximum(0, x)

def sigmoid_derivative(x):
    return x * (1 - x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

# Neural Network Parameters
input_layer_neurons = X_train.shape[1]
output_layer_neurons = y_train.shape[1]

# Hyperparameters
learning_rate = 0.01
epochs = 5000

# Function to train and evaluate the model
def train_and_evaluate(hidden_layer_neurons, activation, activation_derivative):
    # Initialize weights and biases
    np.random.seed(42)
    weights_input_hidden = np.random.uniform(size=(input_layer_neurons, hidden_layer_neurons))
    weights_hidden_output = np.random.uniform(size=(hidden_layer_neurons, output_layer_neurons))
    bias_hidden = np.random.uniform(size=(1, hidden_layer_neurons))
    bias_output = np.random.uniform(size=(1, output_layer_neurons))

    # Training the neural network
    for epoch in range(epochs):
        # Forward propagation
        hidden_layer_input = np.dot(X_train, weights_input_hidden) + bias_hidden
        hidden_layer_output = activation(hidden_layer_input)
        output_layer_input = np.dot(hidden_layer_output, weights_hidden_output) + bias_output
        predicted_output = sigmoid(output_layer_input)

        # Backpropagation
        error = y_train - predicted_output
        d_predicted_output = error * sigmoid_derivative(predicted_output)
        error_hidden_layer = d_predicted_output.dot(weights_hidden_output.T)
        d_hidden_layer = error_hidden_layer * activation_derivative(hidden_layer_output)

        # Update weights and biases
        weights_hidden_output += hidden_layer_output.T.dot(d_predicted_output) * learning_rate
        bias_output += np.sum(d_predicted_output, axis=0, keepdims=True) * learning_rate
        weights_input_hidden += X_train.T.dot(d_hidden_layer) * learning_rate
        bias_hidden += np.sum(d_hidden_layer, axis=0, keepdims=True) * learning_rate

    # Prediction on test data
    hidden_layer_input = np.dot(X_test, weights_input_hidden) + bias_hidden
    hidden_layer_output = activation(hidden_layer_input)
    output_layer_input = np.dot(hidden_layer_output, weights_hidden_output) + bias_output
    predicted_output = sigmoid(output_layer_input)

    # Evaluate the model
    y_test_labels = np.argmax(y_test, axis=1)
    predicted_labels = np.argmax(predicted_output, axis=1)
    accuracy = accuracy_score(y_test_labels, predicted_labels)

    return accuracy

# Test different combinations of hidden layer sizes and activation functions
hidden_layer_sizes = [5, 10, 15]
activation_functions = [(relu, relu_derivative), (sigmoid, sigmoid_derivative)]

results = []

for hidden_layer_neurons in hidden_layer_sizes:
    for activation, activation_derivative in activation_functions:
        accuracy = train_and_evaluate(hidden_layer_neurons, activation, activation_derivative)
        results.append((hidden_layer_neurons, activation.__name__, accuracy))

# Print results
for hidden_layer_neurons, activation_name, accuracy in results:
    print(f'Hidden Layer Neurons: {hidden_layer_neurons}, Activation: {activation_name}, Accuracy: {accuracy * 100:.2f}%')


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Hidden Layer Neurons: 5, Activation: relu, Accuracy: 82.68%
Hidden Layer Neurons: 5, Activation: sigmoid, Accuracy: 81.01%
Hidden Layer Neurons: 10, Activation: relu, Accuracy: 80.45%
Hidden Layer Neurons: 10, Activation: sigmoid, Accuracy: 83.24%
Hidden Layer Neurons: 15, Activation: relu, Accuracy: 84.92%
Hidden Layer Neurons: 15, Activation: sigmoid, Accuracy: 83.24%
